# ==== INTERACTIVE CLUSTERING : EFFICIENCE STUDY ====
> ### DRAFTS : Some examples for report

-----

## [DRAFT] Test usage in PhD Report

In [ ]:
# Définir les données.
dict_of_texts = {
    "0": "Comment signaler un vol de carte bancaire ?",
    "1": "J'ai égaré ma carte bancaire, que faire ?",
    "2": "J'ai perdu ma carte de paiement",
    "3": "Le distributeur a avalé ma carte !",
    "4": "En retirant de l'argent, le GAB a gardé ma carte...",
    "5": "Le distributeur ne m'a pas rendu ma carte bleue.",
    # ...
    "N": "Pourquoi le sans contact ne fonctionne pas ?",
}

In [ ]:
# Import des dépendances.
from cognitivefactory.interactive_clustering.utils.preprocessing import preprocess
from cognitivefactory.interactive_clustering.utils.vectorization import vectorize

# Prétraitement des données.
dict_of_preprocess_texts = preprocess(
    dict_of_texts=dict_of_texts,
    apply_stopwords_deletion=False,
    apply_parsing_filter=False,
    apply_lemmatization=False,
    spacy_language_model="fr_core_news_md",
)
"""
    {"0": "comment signaler un vol de carte bancaire",
     "1": "j ai egare ma carte bancaire, que faire",
     "2": "j ai perdu ma carte de paiement",
     "3": "le distributeur a avale ma carte",
     "4": "en retirant de l argent le gab a garde ma carte",
     "5": "le distributeur ne m a pas rendu ma carte bleue",
     # ...
     "N": "pourquoi le sans contact ne fonctionne pas"}
"""

# Vectorisation des données.
dict_of_vectors = vectorize(
    dict_of_texts=dict_of_preprocess_texts,
    vectorizer_type="tfidf",
)

In [ ]:
# Import des dépendances.
from cognitivefactory.interactive_clustering.constraints.factory import managing_factory

# Création du gestionnaire de contraintes.
constraints_manager = managing_factory(
    manager="binary",
    list_of_data_IDs = list(dict_of_texts.keys()),  # ["0", "1", "2", "3", "4", "5", ..., "N"]
)

# Ajout de contraintes.
constraints_manager.add_constraint(
    data_ID1="0",  # "Comment signaler un vol de carte bancaire ?"
    data_ID2="1",  # "J'ai égaré ma carte bancaire, que faire ?"
    constraint_type="MUST_LINK",
)
constraints_manager.add_constraint(
    data_ID1="3",  # "Le distributeur a avalé ma carte !"
    data_ID2="4",  # "En retirant de l'argent, le GAB a gardé ma carte..."
    constraint_type="MUST_LINK",
)
constraints_manager.add_constraint(
    data_ID1="0",  # "Comment signaler un vol de carte bancaire ?"
    data_ID2="N",  # "Pourquoi le sans contact ne fonctionne pas ?"
    constraint_type="CANNOT_LINK",
)
    # NB: ajouter une contrainte "MUST_LINK" entre "1" et "N" lèverait une erreur.

constraints_manager.get_connected_components()
"""
    [['0', '1'],
     ['2'],
     ['3', '4'],
     ['5'],
     ['N']]
"""

In [ ]:
# Import des dépendances.
from cognitivefactory.interactive_clustering.clustering.factory import clustering_factory

# Initialiser un objet de clustering.
clustering_model = clustering_factory(
    algorithm="kmeans",
    model="COP",
    random_seed=42,
)

# Lancer le clustering.
clustering_result = clustering_model.cluster(
    constraints_manager=constraints_manager,
    nb_clusters=2,
    vectors=dict_of_vectors,
)
"""
    {"0": 0,  # "comment signaler un vol de carte bancaire"
     "1": 0,  # "j ai egare ma carte bancaire, que faire"
     "2": 0,  # "j ai perdu ma carte de paiement"
     "3": 1,  # "le distributeur a avale ma carte"
     "4": 1,  # "en retirant de l argent le gab a garde ma carte"
     "5": 1,  # "le distributeur ne m a pas rendu ma carte bleue"
     # ...
     "N": 1}  # "pourquoi le sans contact ne fonctionne pas"
"""

In [ ]:
# Import des dépendances.
from cognitivefactory.interactive_clustering.sampling.factory import sampling_factory

# Initialiser un objet d'échantillonnage.
sampler = sampling_factory(
    algorithm="random",
    random_seed=42,
)

# Run sampling.
selection = sampler.sample(
    constraints_manager=constraints_manager,
    nb_to_select=2,
    clustering_result=clustering_result,  # optionnel pour "random"
    vectors=dict_of_vectors,  # optionnel pour "random"
)
"""
    [("0", '5"),  # "Comment signaler un vol de carte bancaire ?" vs "Le distributeur ne m'a pas rendu ma carte bleue."
     ("0", '2"),  # "Comment signaler un vol de carte bancaire ?" vs "J'ai perdu ma carte de paiement"
     ("2", 'N")]  # "J'ai perdu ma carte de paiement" vs "Pourquoi le sans contact ne fonctionne pas ?"
"""